<a href="https://colab.research.google.com/github/fleithpi/PROGRAMA-O-ORIENTADA-A-OBJETOS/blob/main/Biblioteca_Verificador_de_atrasos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
# --- 1. CLASSE LIVRO ATUALIZADA (Com Categoria e Ano) ---
from datetime import datetime, timedelta
import csv


class Livro:
    def __init__(self, titulo: str, autor: str, categoria: str = "Geral", ano_publicacao: int = None):
        self.__titulo = titulo.strip()
        self.__autor = autor.strip()
        self.__categoria = categoria.strip()
        self.__ano_publicacao = ano_publicacao
        self.__disponivel = True

    @property
    def titulo(self) -> str:
        return self.__titulo

    @property
    def autor(self) -> str:
        return self.__autor

    @property
    def categoria(self) -> str:
        return self.__categoria

    @property
    def ano_publicacao(self) -> int:
        return self.__ano_publicacao

    @property
    def disponivel(self) -> bool: return self.__disponivel
    def is_disponivel(self) -> bool: return self.__disponivel

    # Setters para Gerenciamento
    def set_categoria(self, nova_categoria: str):self.__categoria = nova_categoria.strip()


    def set_ano_publicacao(self, novo_ano: int):
        self.__ano_publicacao = novo_ano

    def emprestar(self) -> bool:
        if self.__disponivel:
            self.__disponivel = False
            return True
        return False # A mensagem de erro é tratada na Biblioteca

    def devolver(self) -> bool:
        if not self.__disponivel:
            self.__disponivel = True
            return True
        return False

    def __str__(self):
        status = "Disponível" if self.disponivel else "Emprestado"
        ano_str = f" ({self.ano_publicacao})" if self.ano_publicacao else ""
        return f"({self.titulo}' por {self.autor}{ano_str} | Cat: {self.categoria})"

In [31]:
class Usuario:
    def __init__(self, id_usuario: int, nome: str):
        self.__id = id_usuario
        self.__nome = nome.strip()

    @property
    def id(self) -> int:
        return self.__id

    @property
    def nome(self) -> str:
        return self.__nome

    def __str__(self):
        return f"Usuário: {self.nome} (ID: {self.id})"

    def __eq__(self, outro):
        if insnstance(outro, Usuario): return self.id == outro.id
        return False
    def __str___(self): return f"ID{self.id}, Nome: {self.nome}"


In [32]:


class Biblioteca:
    def __init__(self, dias_max_emprestimos: int=7):

        self.__acervo = {}

        self.__prazo_dias = dias_max_emprestimos

        self.__usuarios: dict[int, Usuario] = {}

        self.__emprestimos: dict[str, tuple[int, datetime.date]] = {}


    # --- MÉTODOS DE MANUTENÇÃO ---
    def adicionar_livro(self, livro: Livro):
        titulo = livro.titulo
        if titulo in self.__acervo:
            print(f"ATENÇÃO: Livro '{titulo}' já está no acervo.")
        else:
            self.__acervo[titulo] = livro
            print(f"SUCESSO: Livro '{titulo}' adicionado ao acervo.")

    def adicionar_usuario(self, usuario: Usuario):
        user_id = usuario.id
        if user_id in self.__usuarios:
            print(f"ATENÇÃO: Usuário ID {user_id} já está registrado.")
        else:
            self.__usuarios[user_id] = usuario
            print(f"SUCESSO: Usuário '{usuario.nome}' (ID: {user_id}) registrado.")

    # --- NOVO MÉTODO DE GERENCIAMENTO ---
    def gerenciar_livro(self, titulo_livro: str, nova_categoria: str = None, novo_ano: int = None):
        """Atualiza a categoria e/ou o ano de publicação de um livro."""

        if titulo_livro not in self.__acervo:
            print(f"ERRO: Livro '{titulo_livro}' não encontrado no acervo para gerenciamento.")
            return

        livro = self.__acervo[titulo_livro]
        alterado = False

        if nova_categoria is not None and nova_categoria.strip():
            livro.set_categoria(nova_categoria)
            print(f"SUCESSO: Categoria de '{titulo_livro}' atualizada para '{livro.categoria}'.")
            alterado = True

        if novo_ano is not None:
            try:
                novo_ano = int(novo_ano)
                livro.set_ano_publicacao(novo_ano)
                print(f"SUCESSO: Ano de publicação de '{titulo_livro}' atualizado para {livro.ano_publicacao}.")
                alterado = True
            except ValueError:
                print(f"ERRO: O ano fornecido ('{novo_ano}') não é um número inteiro válido.")
                return

        if not alterado:
            print(f"ATENÇÃO: Nenhuma alteração foi solicitada para '{titulo_livro}'.")

    # --- MÉTODOS DE EMPRÉSTIMO E DEVOLUÇÃO (Com Rastreamento) ---
    def emprestar_livro(self, id_usuario: int, titulo_livro: str):
        if id_usuario not in self.__usuarios:
            print(f"ERRO: Usuário ID {id_usuario} não encontrado.")
            return
        if titulo_livro not in self.__acervo:
            print(f"ERRO: Livro '{titulo_livro}' não encontrado no acervo.")
            return

        livro = self.__acervo[titulo_livro]
        usuario = self.__usuarios[id_usuario]

        if livro.emprestar():
            hoje = datetime.now().date()
            data_devolucao = hoje + timedelta(days=self.__prazo_dias)

            self.__emprestimos[titulo_livro] = (id_usuario, data_devolucao)

            print(f"EMPRÉSTIMO SUCESSO: '{livro.titulo}' emprestado para {usuario.nome}.")
            print(f"  -> Deve ser devolvido até: {data_devolucao.strftime('%d/%m/%Y')}")

        else:
            # Correção de NameError/lógica:
            # Se o livro.emprestar() falhou, ele está na verdade emprestado.
            if titulo_livro in self.__emprestimos:
                id_quem_pegou, data_devolucao = self.__emprestimos[titulo_livro]
                nome_quem_pegou = self.__usuarios.get(id_quem_pegou).nome if id_quem_pegou in self.__usuarios else "usuário desconhecido"
                print(f"EMPRÉSTIMO FALHA: '{livro.titulo}' já está emprestado para {nome_quem_pegou}.")
            else:
                # Caso extremo: livro não disponível, mas não rastreado (erro interno)
                 print(f"EMPRÉSTIMO FALHA: '{livro.titulo}' está indisponível (Erro de rastreio).")


    def devolver_livro(self, id_usuario: int, titulo_livro: str):
        if titulo_livro not in self.__acervo:
            print(f"ERRO: Livro '{titulo_livro}' não encontrado para devolução.")
            return

        emprestimo_info = self.__emprestimos.get(titulo_livro)
        if emprestimo_info is None:
            # Se não está em self.__emprestimos, o livro pode já estar disponível
            if self.__acervo[titulo_livro].disponivel:
                print(f"DEVOLUÇÃO FALHA: '{titulo_livro}' já estava disponível.")
            else:
                print(f"DEVOLUÇÃO FALHA: '{titulo_livro}' está indisponível, mas sem rastreio de empréstimo (Erro interno).")
            return

        # Desempacota a tupla do empréstimo
        id_quem_pegou, data_devolucao = emprestimo_info

        if id_quem_pegou != id_usuario:
             usuario_esperado = self.__usuarios.get(id_quem_pegou).nome if id_quem_pegou in self.__usuarios else "usuário desconhecido"
             print(f"ERRO DE DEVOLUÇÃO: O livro '{titulo_livro}' foi emprestado para {usuario_esperado} (ID {id_quem_pegou}), e não para o ID {id_usuario}.")
             return

        livro = self.__acervo[titulo_livro]
        if livro.devolver():
            del self.__emprestimos[titulo_livro]

            # Verifica se houve atraso
            hoje = datetime.now().date()
            usuario_nome = self.__usuarios[id_usuario].nome

            if hoje > data_devolucao:
                dias_atraso = (hoje - data_devolucao).days
                print(f"DEVOLUÇÃO COM ATRASO: '{livro.titulo}' devolvido por {usuario_nome} ({dias_atraso} dias de atraso).")
            else:
                print(f"DEVOLUÇÃO SUCESSO: '{livro.titulo}' devolvido à biblioteca por {usuario_nome}.")
        else:
             # Isso só deve acontecer se livro.devolver() falhar, o que é improvável dado o check acima, mas é boa prática
             print(f"DEVOLUÇÃO FALHA: Livro '{titulo_livro}' já estava marcado como disponível no objeto Livro.")


    # --- MÉTODOS DE VISUALIZAÇÃO (Corrigidos) ---

    # CORREÇÃO PRINCIPAL: Remover a variável 'ordenar_por' não definida.
    def listar_acervo(self):
        """Chama listar_livros usando o padrão 'titulo'."""
        self.listar_livros()

    def listar_livros(self, ordenar_por: str = 'titulo'):
        """Lista todos os livros no acervo com seus status."""
        print("\n--- LIVROS NO ACERVO ---")

        if not self.__acervo:
            print("O acervo está vazio.")
            return

        livros_lista = list(self.__acervo.values())

        # Lógica de ordenação
        try:
            if ordenar_por == 'autor':
                livros_lista.sort(key=lambda livro: livro.autor.lower())
            elif ordenar_por == 'ano':
                # Ordena por ano (coloca None/vazios no final)
                livros_lista.sort(key=lambda livro: livro.ano_publicacao if livro.ano_publicacao is not None else float('inf'))
            elif ordenar_por == 'categoria':
                livros_lista.sort(key=lambda livro: livro.categoria.lower())
            else: # PADRÃO É TITULO
                livros_lista.sort(key=lambda livro: livro.titulo.lower())
        except AttributeError as e:
            print(f"ERRO AO ORDENAR: {e}")
            return

        hoje = datetime.now().date()

        # CORREÇÃO: Apenas um loop de listagem.
        for livro in livros_lista:
            # Usa o __str__ do Livro, que já fornece Título/Autor/Status
            status_completo = str(livro)

            # Busca informações de empréstimo (ID e Data)
            if livro.titulo in self.__emprestimos:
                id_usuario, data_devolucao = self.__emprestimos[livro.titulo]
                nome_usuario = self.__usuarios.get(id_usuario, Usuario(-1, "Desconhecido")).nome

                status_completo += f" (Com: {nome_usuario} | Devolução: {data_devolucao.strftime('%d/%m/%Y')})"

                # Adiciona flag de atraso
                if data_devolucao < hoje:
                    status_completo += " [ATRASADO!]"

            print(f"- {status_completo}")

    def lista_atrasos(self):
      # LISTA OS EMPRESTIMOS COM ATRASO
        print("\n--- VERIFICADOR DE ATRASOS ---")
        hoje = datetime.now().date()
        atrasados_encontrados = False

        for titulo, (id_usuario, data_devolucao) in self.__emprestimos.items():
            if data_devolucao < hoje:
                atrasados_encontrados = True
                livro = self.__acervo.get(titulo)
                usuario = self.__usuarios.get(id_usuario, Usuario(-1, "Desconhecido"))
                dias_atraso = (hoje - data_devolucao).days

                print(f"[ATRASADO] '{livro.titulo}' com {usuario.nome} (ID: {usuario.id})")
                print(f"  -> Devolução esperada: {data_devolucao.strftime('%d/%m/%Y')} ({dias_atraso} dias de atraso)")

        if not atrasados_encontrados:
            print("Nenhum livro com atraso no momento.")

    def exportar_acervo_csv(self, nome_arquivo: str = "acervo_biblioteca.csv"):
        """NOVO: Exporta o status completo do acervo para um arquivo CSV."""
        print(f"\nExportando acervo para '{nome_arquivo}'...")
        if not self.__acervo:
            print("ERRO: Acervo está vazio. Nada para exportar.")
            return

        headers = [
            "Titulo", "Autor", "Ano", "Categoria",
            "Status", "ID_Usuario", "Nome_Usuario", "Data_Devolucao"
        ]

        hoje = datetime.now().date()

        try:
            with open(nome_arquivo, 'w', newline='', encoding='utf-8') as f:
                writer = csv.DictWriter(f, fieldnames=headers)
                writer.writeheader()

                for livro in self.__acervo.values():
                    row = {
                        "Titulo": livro.titulo,
                        "Autor": livro.autor,
                        "Ano": livro.ano_publicacao,
                        "Categoria": livro.categoria,
                        "ID_Usuario": "", "Nome_Usuario": "", "Data_Devolucao": ""
                    }

                    if livro.is_disponivel():
                        row["Status"] = "Disponível"
                    else:
                        if livro.titulo in self.__emprestimos:
                            id_usuario, data_devolucao = self.__emprestimos[livro.titulo]
                            usuario = self.__usuarios.get(id_usuario, Usuario(-1, "Desconhecido"))

                            row["Status"] = "Emprestado"
                            row["ID_Usuario"] = usuario.id
                            row["Nome_Usuario"] = usuario.nome
                            row["Data_Devolucao"] = data_devolucao.strftime('%Y-%m-%d')

                            if data_devolucao < hoje:
                                row["Status"] = "Emprestado (Atrasado)"
                        else:
                            row["Status"] = "Indisponível (Erro de Rastreio)"

                    writer.writerow(row)

            print(f"SUCESSO: Acervo exportado para '{nome_arquivo}'.")

        except IOError as e:
            print(f"ERRO: Falha ao escrever o arquivo CSV: {e}")
        except Exception as e:
            print(f"ERRO: Ocorreu um erro inesperado durante a exportação: {e}")




In [43]:
if __name__ == '__main__':
    # ... (Seção de teste da biblioteca principal: Adicionar, Empréstimo, Devolução, Gerenciamento)

    # Reutilizando user_a e user_b definidos anteriormente (IDs 10 e 20)
    user_a = Usuario(10, "Maria")
    user_b = Usuario(20, "João")

    # NOVO: Adicionando livros com categoria e ano
    print("\n--- TESTE DE GERENCIAMENTO DE LIVROS ---")
    # ... (código de gerenciamento)

    # --- TESTE DE ATRASOS (Com simulação de tempo) ---
    print("\n--- TESTE DE ATRASOS ---")
    biblioteca_teste = Biblioteca(dias_max_emprestimos=3)

    # Usando os IDs 10 e 20
    biblioteca_teste.adicionar_usuario(user_a) # ID 10
    biblioteca_teste.adicionar_usuario(user_b) # ID 20

    # CORREÇÃO: Cria e adiciona os livros de teste de atraso
    l1 = Livro("O Livro Atrasado de Teste", "Autor Atraso")
    l2 = Livro("O Livro No Prazo de Teste", "Autor Prazo")
    biblioteca_teste.adicionar_livro(l1)
    biblioteca_teste.adicionar_livro(l2)

    # Empréstimo do livro Atrasado (Usando ID 10)
    biblioteca_teste.emprestar_livro(user_a.id, l1.titulo) # ID 10, "O Livro Atrasado de Teste"

    # SIMULAÇÃO DE ATRASO: Força a data de devolução para o passado (10 dias atrás)
    # Acessa o dicionário de empréstimos e substitui a data de devolução limite.
    # OBS: O acesso a __emprestimos é para fins de teste e deve usar o nome interno: _Biblioteca__emprestimos
    titulo_atrasado = l1.titulo
    try:
        id_temp, data_limite_original = biblioteca_teste._Biblioteca__emprestimos[titulo_atrasado]
        data_atrasada = datetime.now().date() - timedelta(days=10)
        biblioteca_teste._Biblioteca__emprestimos[titulo_atrasado] = (id_temp, data_atrasada)
    except KeyError:
        print(f"ERRO DE SIMULAÇÃO: O livro '{titulo_atrasado}' não foi encontrado no rastreio de empréstimos.")

    # Empréstimo do livro No Prazo (Usando ID 20)
    biblioteca_teste.emprestar_livro(user_b.id, l2.titulo) # ID 20, "O Livro No Prazo de Teste"

    # 1. Lista atrasos (deve pegar O Livro Atrasado de Teste)
    biblioteca_teste.lista_atrasos()

    # 2. Devolve o livro Atrasado (Usando ID 10)
    biblioteca_teste.devolver_livro(user_a.id, l1.titulo)

    # 3. Devolve o livro No Prazo (Usando ID 20)
    biblioteca_teste.devolver_livro(user_b.id, l2.titulo)

    # 4. Verifica atrasos novamente (deve estar vazio)
    biblioteca_teste.lista_atrasos()

    # Exporta CSV com status atual
    # biblioteca_teste.exportar_acervo_csv("acervo_final_teste.csv") # Renomeado para evitar conflito com o anterior


--- TESTE DE GERENCIAMENTO DE LIVROS ---

--- TESTE DE ATRASOS ---
SUCESSO: Usuário 'Maria' (ID: 10) registrado.
SUCESSO: Usuário 'João' (ID: 20) registrado.
SUCESSO: Livro 'O Livro Atrasado de Teste' adicionado ao acervo.
SUCESSO: Livro 'O Livro No Prazo de Teste' adicionado ao acervo.
EMPRÉSTIMO SUCESSO: 'O Livro Atrasado de Teste' emprestado para Maria.
  -> Deve ser devolvido até: 26/10/2025
EMPRÉSTIMO SUCESSO: 'O Livro No Prazo de Teste' emprestado para João.
  -> Deve ser devolvido até: 26/10/2025

--- VERIFICADOR DE ATRASOS ---
[ATRASADO] 'O Livro Atrasado de Teste' com Maria (ID: 10)
  -> Devolução esperada: 13/10/2025 (10 dias de atraso)
DEVOLUÇÃO COM ATRASO: 'O Livro Atrasado de Teste' devolvido por Maria (10 dias de atraso).
DEVOLUÇÃO SUCESSO: 'O Livro No Prazo de Teste' devolvido à biblioteca por João.

--- VERIFICADOR DE ATRASOS ---
Nenhum livro com atraso no momento.
